# **Jour 5 : Introduction Docker**

## **Objectifs du jour 5**

Comprendre les concepts fondamentaux de Docker, créer votre premier Dockerfile pour une application Python, containeriser votre pipeline de données, et intégrer Docker dans votre workflow de développement.

## **1. Concepts fondamentaux de Docker**

### **Comprendre Docker par l'analogie du déménagement**

Docker résout le problème du "ça marche sur ma machine" de la même façon qu'un déménageur professionnel résout le problème du transport de meubles. Imaginez que vous déménagiez et que vous vouliez être sûr que votre bureau fonctionne exactement pareil dans votre nouvelle maison.

**Sans Docker (déménagement traditionnel) :** Vous démontez votre bureau, transportez les pièces séparément, et tentez de le remonter dans la nouvelle maison. Problème : la nouvelle maison a des prises électriques différentes, des dimensions de pièces différentes, et vous avez perdu quelques vis en route. Votre bureau ne fonctionne plus pareil.

**Avec Docker (container de déménagement) :** Vous mettez votre bureau complet dans un container hermétique avec tout ce dont il a besoin : alimentation électrique, éclairage, même l'air conditionné. Le container arrive dans la nouvelle maison et votre bureau fonctionne instantanément, exactement comme avant.

**Application au développement :**

Votre application Python est comme votre bureau. Elle a besoin d'une version spécifique de Python (l'électricité), de packages particuliers (les meubles), et de configurations précises (l'agencement). Docker emballe tout cela dans un container qui fonctionne identiquement sur votre machine de développement, celle de votre collègue, et sur les serveurs de production.

### **Différence entre Image et Container**

**Image Docker = Plan de construction :** Une image Docker est comme un plan d'architecte détaillé. Elle contient toutes les instructions pour construire votre environnement : quelle version de Python installer, quels packages ajouter, comment configurer l'application. L'image est statique, elle ne change pas.

**Container Docker = Maison construite :** Un container est une instance en cours d'exécution d'une image. C'est comme une maison construite à partir du plan. Vous pouvez avoir plusieurs maisons (containers) construites à partir du même plan (image), chacune avec ses propres habitants (processus) et ses propres affaires (données).

In [ ]:
# Analogie avec les commandes Docker
# docker build -t mon-app .          # Créer le plan (image)
# docker run mon-app                 # Construire une maison (container)
# docker run mon-app                 # Construire une deuxième maison identique

print("Commandes Docker de base :")
print("docker build -t mon-app .    # Créer une image")
print("docker run mon-app           # Lancer un container")
print("docker ps                    # Voir les containers actifs")
print("docker images                # Voir les images disponibles")

### **Avantages concrets pour Data Engineering**

**Reproductibilité garantie :** Votre pipeline de données fonctionne avec pandas 2.0.1, numpy 1.24.3, et Python 3.11.2. Docker garantit que ces versions exactes seront utilisées partout, éliminant les bugs liés aux différences d'environnement.

**Isolation complète :** Votre pipeline peut utiliser PostgreSQL 15 pendant qu'un autre projet sur le même serveur utilise PostgreSQL 12. Docker isole complètement les environnements.

**Déploiement simplifié :** Plus besoin d'installer Python, configurer les dépendances, et espérer que tout fonctionne sur le serveur de production. Vous déployez un container qui contient déjà tout.

**Collaboration facilitée :** Un nouveau développeur peut lancer votre projet en 30 secondes avec `docker run`, sans passer des heures à configurer son environnement.

## **2. Création de votre premier Dockerfile**

### **Dockerfile pour application Data Engineering**

Un Dockerfile est la recette pour construire votre image Docker. Chaque instruction dans le Dockerfile ajoute une couche à votre image, comme empiler des couches de gâteau.

In [ ]:
# Créons un exemple de Dockerfile pour une application Data Engineering
dockerfile_content = """
# Dockerfile pour pipeline de données
# Commentaires expliquent chaque étape pour l'apprentissage

# Étape 1: Choisir l'image de base
FROM python:3.11-slim

# Pourquoi python:3.11-slim ?
# - Version officielle Python maintenue par l'équipe Docker
# - "slim" = version allégée sans packages inutiles
# - Plus petite et plus sécurisée que l'image complète

# Étape 2: Métadonnées de l'image
LABEL maintainer="votre.email@example.com"
LABEL description="Pipeline de données pour analyse des ventes"
LABEL version="1.0.0"

# Étape 3: Variables d'environnement pour optimiser Python
ENV PYTHONUNBUFFERED=1 \\
    PYTHONDONTWRITEBYTECODE=1 \\
    PIP_NO_CACHE_DIR=1 \\
    PIP_DISABLE_PIP_VERSION_CHECK=1

# Explication des variables :
# PYTHONUNBUFFERED=1 : Affichage immédiat des logs (crucial pour Docker)
# PYTHONDONTWRITEBYTECODE=1 : Pas de fichiers .pyc (réduit la taille)
# PIP_NO_CACHE_DIR=1 : Pas de cache pip (économise l'espace)

# Étape 4: Installation des dépendances système
RUN apt-get update && apt-get install -y --no-install-recommends \\
    curl \\
    build-essential \\
    && rm -rf /var/lib/apt/lists/* \\
    && apt-get clean

# Étape 5: Création d'un utilisateur non-root (sécurité)
RUN groupadd --gid 1000 datauser && \\
    useradd --uid 1000 --gid datauser --shell /bin/bash --create-home datauser

# Étape 6: Installation de Poetry
RUN pip install poetry==1.7.1

# Étape 7: Configuration Poetry pour Docker
ENV POETRY_NO_INTERACTION=1 \\
    POETRY_VENV_IN_PROJECT=1 \\
    POETRY_CACHE_DIR=/tmp/poetry_cache

# Étape 8: Définition du répertoire de travail
WORKDIR /app

# Étape 9: Copie des fichiers de dépendances AVANT le code
COPY pyproject.toml poetry.lock ./

# Étape 10: Installation des dépendances
RUN poetry install --only=main && rm -rf $POETRY_CACHE_DIR

# Étape 11: Copie du code source
COPY src/ ./src/
COPY scripts/ ./scripts/
COPY data/ ./data/

# Étape 12: Changement des permissions et utilisateur
RUN chown -R datauser:datauser /app
USER datauser

# Étape 13: Port exposé (si votre app a une interface web)
EXPOSE 8000

# Étape 14: Health check pour monitoring
HEALTHCHECK --interval=30s --timeout=10s --start-period=5s --retries=3 \\
    CMD python -c "import src; print('OK')" || exit 1

# Étape 15: Commande par défaut
CMD ["poetry", "run", "python", "-m", "src.main"]
"""

# Sauvegarder le Dockerfile
with open('Dockerfile', 'w') as f:
    f.write(dockerfile_content)

print("Dockerfile créé avec succès !")
print("\nStructure du Dockerfile :")
print("- Image de base : python:3.11-slim")
print("- Optimisations Python pour Docker")
print("- Utilisateur non-root pour la sécurité")
print("- Poetry pour la gestion des dépendances")
print("- Health check intégré")

### **Construction et test de votre image**

In [ ]:
# Commandes pour construire et tester l'image Docker
print("Commandes de construction :")
print("docker build -t pipeline-ventes:latest .")
print("docker build -t pipeline-ventes:v1.0 --progress=plain .")
print("\nVérification de l'image :")
print("docker images | grep pipeline-ventes")
print("\nCommandes de test :")
print("docker run --rm pipeline-ventes:latest")
print("docker run -it --rm pipeline-ventes:latest /bin/bash")
print("docker run --rm -v $(pwd)/data:/app/data pipeline-ventes:latest")
print("\nCommandes de debugging :")
print("docker logs mon-pipeline")
print("docker exec -it mon-pipeline /bin/bash")
print("docker inspect mon-pipeline")
print("docker stats mon-pipeline")

## **3. Containerisation d'une application Python complète**

### **Projet pratique : Pipeline de données containerisé**

Créons ensemble un pipeline de données complet qui traite des fichiers CSV, applique des transformations, et génère des rapports.

In [ ]:
# Créons la structure du projet
import os
from pathlib import Path

# Structure du projet
project_structure = {
    'src': ['__init__.py', 'main.py'],
    'src/pipeline': ['__init__.py', 'extractor.py', 'transformer.py', 'loader.py'],
    'src/utils': ['__init__.py', 'logger.py'],
    'data/input': [],
    'data/processed': [],
    'data/output': [],
    'tests': [],
    'scripts': []
}

# Créer la structure de dossiers
for folder, files in project_structure.items():
    Path(folder).mkdir(parents=True, exist_ok=True)
    for file in files:
        (Path(folder) / file).touch()

print("Structure du projet créée :")
for folder in project_structure.keys():
    print(f"📁 {folder}/")
    for file in project_structure[folder]:
        print(f"   📄 {file}")

In [ ]:
# Code du pipeline principal - src/main.py
main_py_content = '''
"""Point d'entrée du pipeline de données containerisé."""

import logging
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# Configuration du logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

def create_demo_data(input_dir: Path) -> None:
    """Crée des données de démonstration."""
    # Génération de données de ventes fictives
    np.random.seed(42)
    dates = pd.date_range('2024-01-01', periods=1000, freq='D')
    
    demo_data = pd.DataFrame({
        'date': np.random.choice(dates, 1000),
        'product_id': np.random.randint(1, 101, 1000),
        'product_name': [f"Produit_{i}" for i in np.random.randint(1, 101, 1000)],
        'quantity': np.random.randint(1, 50, 1000),
        'unit_price': np.round(np.random.uniform(10, 500, 1000), 2),
        'customer_id': [f"CUST_{i:04d}" for i in np.random.randint(1, 501, 1000)],
        'region': np.random.choice(['Nord', 'Sud', 'Est', 'Ouest'], 1000)
    })
    
    demo_file = input_dir / "sales_demo.csv"
    demo_data.to_csv(demo_file, index=False)
    print(f"📝 Fichier de démonstration créé : {demo_file}")

def main() -> int:
    """Exécute le pipeline de données complet."""
    logger = logging.getLogger("pipeline")
    
    try:
        logger.info("🚀 Démarrage du pipeline de données")
        
        # Configuration des chemins (compatibles Docker)
        base_dir = Path(".")
        input_dir = base_dir / "data" / "input"
        processed_dir = base_dir / "data" / "processed"
        output_dir = base_dir / "data" / "output"
        
        # Création des dossiers si nécessaire
        for directory in [input_dir, processed_dir, output_dir]:
            directory.mkdir(parents=True, exist_ok=True)
            logger.info(f"📁 Dossier vérifié : {directory}")
        
        # Vérification des fichiers d'entrée
        input_files = list(input_dir.glob("*.csv"))
        if not input_files:
            logger.warning("⚠️  Aucun fichier CSV trouvé dans data/input/")
            logger.info("💡 Création d'un fichier de démonstration")
            create_demo_data(input_dir)
            input_files = list(input_dir.glob("*.csv"))
        
        # Traitement de chaque fichier
        for input_file in input_files:
            logger.info(f"📊 Traitement de {input_file.name}")
            
            # Lecture des données
            raw_data = pd.read_csv(input_file)
            logger.info(f"✅ Extraction : {len(raw_data)} lignes")
            
            # Transformation simple
            clean_data = raw_data.dropna().drop_duplicates()
            clean_data['total_amount'] = clean_data['quantity'] * clean_data['unit_price']
            clean_data['processed_at'] = pd.Timestamp.now()
            logger.info(f"✅ Transformation : {len(clean_data)} lignes")
            
            # Sauvegarde
            output_file = output_dir / f"processed_{input_file.stem}.csv"
            clean_data.to_csv(output_file, index=False)
            logger.info(f"✅ Sauvegarde : {output_file.name}")
        
        logger.info("🎉 Pipeline terminé avec succès")
        return 0
        
    except Exception as e:
        logger.error(f"❌ Erreur dans le pipeline : {e}")
        return 1

if __name__ == "__main__":
    sys.exit(main())
'''

# Sauvegarder le fichier main.py
with open('src/main.py', 'w') as f:
    f.write(main_py_content)

print("✅ Fichier src/main.py créé")
print("\nFonctionnalités du pipeline :")
print("- Création automatique de données de démonstration")
print("- Lecture et traitement des fichiers CSV")
print("- Nettoyage et enrichissement des données")
print("- Sauvegarde des résultats")
print("- Logging détaillé")

In [ ]:
# Testons le pipeline localement
import subprocess
import sys

# Installer les dépendances nécessaires
try:
    import pandas as pd
    import numpy as np
    print("✅ Dépendances déjà installées")
except ImportError:
    print("Installation des dépendances...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas", "numpy"])

# Exécuter le pipeline
print("\n🚀 Exécution du pipeline...")
exec(open('src/main.py').read())

# Vérifier les résultats
output_files = list(Path('data/output').glob('*.csv'))
print(f"\n📊 Fichiers générés : {len(output_files)}")
for file in output_files:
    print(f"   📄 {file.name}")

### **Docker Compose pour environnement complet**

Pour un projet plus complexe, utilisez Docker Compose pour orchestrer plusieurs services.

In [ ]:
# Créons un fichier docker-compose.yml
docker_compose_content = '''
version: '3.8'

services:
  # Application principale
  pipeline:
    build: .
    container_name: data-pipeline
    volumes:
      # Montage des données pour persistance
      - ./data:/app/data
      - ./logs:/app/logs
    environment:
      - LOG_LEVEL=INFO
      - ENVIRONMENT=development
    depends_on:
      - database
      - redis
    networks:
      - data-network
    restart: unless-stopped

  # Base de données PostgreSQL
  database:
    image: postgres:15-alpine
    container_name: postgres-db
    environment:
      POSTGRES_DB: datawarehouse
      POSTGRES_USER: datauser
      POSTGRES_PASSWORD: datapass123
    volumes:
      - postgres_data:/var/lib/postgresql/data
    ports:
      - "5432:5432"
    networks:
      - data-network
    restart: unless-stopped

  # Redis pour cache et queues
  redis:
    image: redis:7-alpine
    container_name: redis-cache
    command: redis-server --appendonly yes
    volumes:
      - redis_data:/data
    ports:
      - "6379:6379"
    networks:
      - data-network
    restart: unless-stopped

  # Interface d'administration
  adminer:
    image: adminer:latest
    container_name: db-admin
    ports:
      - "8080:8080"
    depends_on:
      - database
    networks:
      - data-network
    restart: unless-stopped

# Volumes persistants
volumes:
  postgres_data:
  redis_data:

# Réseau pour communication inter-services
networks:
  data-network:
    driver: bridge
'''

# Sauvegarder le fichier docker-compose.yml
with open('docker-compose.yml', 'w') as f:
    f.write(docker_compose_content)

print("✅ Fichier docker-compose.yml créé")
print("\nServices inclus :")
print("🐳 pipeline - Application principale")
print("🐘 database - PostgreSQL")
print("🔴 redis - Cache et queues")
print("🔧 adminer - Interface d'administration DB")
print("\nCommandes utiles :")
print("docker-compose up -d        # Lancer tous les services")
print("docker-compose logs -f      # Voir les logs")
print("docker-compose down         # Arrêter tous les services")

## **4. Intégration Docker dans le workflow de développement**

### **Workflow de développement avec Docker**

In [ ]:
# Créons des scripts pour faciliter le développement avec Docker
import os

# Script de développement
dev_script = '''
#!/bin/bash
# scripts/dev-docker.sh
# Script pour développement avec Docker

set -e

echo "🐳 Démarrage de l'environnement de développement Docker"

# Construction de l'image de développement
docker build -f Dockerfile.dev -t pipeline-dev .

# Lancement avec montage des sources
docker run -it --rm \\
  --name pipeline-dev \\
  -v $(pwd)/src:/app/src \\
  -v $(pwd)/tests:/app/tests \\
  -v $(pwd)/data:/app/data \\
  -p 8000:8000 \\
  pipeline-dev

echo "✅ Environnement de développement prêt"
'''

# Script de test
test_script = '''
#!/bin/bash
# scripts/test-docker.sh
# Tests dans un environnement Docker propre

echo "🧪 Exécution des tests dans Docker"

# Construction de l'image de test
docker build -t pipeline-test .

# Exécution des tests
docker run --rm \\
  -v $(pwd)/tests:/app/tests \\
  -v $(pwd)/data:/app/data \\
  pipeline-test \\
  python -m pytest tests/ -v

echo "✅ Tests terminés"
'''

# Créer le dossier scripts s'il n'existe pas
os.makedirs('scripts', exist_ok=True)

# Sauvegarder les scripts
with open('scripts/dev-docker.sh', 'w') as f:
    f.write(dev_script)

with open('scripts/test-docker.sh', 'w') as f:
    f.write(test_script)

# Rendre les scripts exécutables (sur Unix)
try:
    os.chmod('scripts/dev-docker.sh', 0o755)
    os.chmod('scripts/test-docker.sh', 0o755)
except:
    pass  # Windows n'a pas chmod

print("✅ Scripts de développement créés")
print("📁 scripts/dev-docker.sh - Environnement de développement")
print("📁 scripts/test-docker.sh - Tests automatisés")
print("\nUsage :")
print("./scripts/dev-docker.sh   # Développement")
print("./scripts/test-docker.sh  # Tests")

### **Optimisation des images Docker**

In [ ]:
# Dockerfile multi-stage pour production
dockerfile_prod = '''
# Dockerfile.prod - Version optimisée production
# Stage 1: Builder
FROM python:3.11-slim as builder

ENV POETRY_NO_INTERACTION=1 \\
    POETRY_VENV_IN_PROJECT=1 \\
    POETRY_CACHE_DIR=/tmp/poetry_cache

WORKDIR /app

# Installation Poetry
RUN pip install poetry

# Installation des dépendances
COPY pyproject.toml poetry.lock ./
RUN poetry install --only=main && rm -rf $POETRY_CACHE_DIR

# Stage 2: Production
FROM python:3.11-slim as production

# Métadonnées
LABEL maintainer="votre.email@example.com"
LABEL description="Pipeline de données - Production"

# Variables d'environnement optimisées
ENV PYTHONUNBUFFERED=1 \\
    PYTHONDONTWRITEBYTECODE=1 \\
    PATH="/app/.venv/bin:$PATH"

# Installation des dépendances système minimales
RUN apt-get update && apt-get install -y --no-install-recommends \\
    curl \\
    && rm -rf /var/lib/apt/lists/* \\
    && apt-get clean

# Création utilisateur non-root
RUN groupadd --gid 1000 appuser && \\
    useradd --uid 1000 --gid appuser --shell /bin/bash --create-home appuser

WORKDIR /app

# Copie de l'environnement virtuel depuis le builder
COPY --from=builder /app/.venv /app/.venv

# Copie du code source
COPY --chown=appuser:appuser src/ ./src/
COPY --chown=appuser:appuser scripts/ ./scripts/

# Changement vers utilisateur non-root
USER appuser

# Health check
HEALTHCHECK --interval=30s --timeout=10s --start-period=5s --retries=3 \\
    CMD python -c "import src; print('OK')" || exit 1

# Commande par défaut
CMD ["python", "-m", "src.main"]
'''

# Sauvegarder le Dockerfile de production
with open('Dockerfile.prod', 'w') as f:
    f.write(dockerfile_prod)

print("✅ Dockerfile de production créé")
print("\nOptimisations incluses :")
print("🏗️  Build multi-stage pour réduire la taille")
print("👤 Utilisateur non-root pour la sécurité")
print("🩺 Health check intégré")
print("⚡ Variables d'environnement optimisées")
print("\nComparaison typique des tailles :")
print("📊 Image de développement : ~1.2GB")
print("📊 Image de production : ~200MB")

## **Exercices pratiques**

### **Exercice 1 : Créer et tester votre premier container**

In [ ]:
# Exercice 1 : Créer un simple pipeline de données
print("🎯 Exercice 1 : Créer et tester votre premier container")
print("\nÉtapes à suivre :")
print("1. Créer un fichier requirements.txt avec pandas et numpy")
print("2. Créer un script Python simple qui lit un CSV")
print("3. Créer un Dockerfile pour containeriser le script")
print("4. Construire l'image Docker")
print("5. Exécuter le container avec des données de test")

# Créer requirements.txt
requirements = """
pandas==2.0.1
numpy==1.24.3
"""

with open('requirements.txt', 'w') as f:
    f.write(requirements.strip())

print("\n✅ requirements.txt créé")
print("\nCommandes à exécuter :")
print("docker build -t mon-premier-pipeline .")
print("docker run --rm -v $(pwd)/data:/app/data mon-premier-pipeline")

### **Exercice 2 : Pipeline avec Docker Compose**

In [ ]:
# Exercice 2 : Créer un environnement complet avec Docker Compose
print("🎯 Exercice 2 : Pipeline avec Docker Compose")
print("\nObjectif : Créer un pipeline qui :")
print("- Lit des données depuis PostgreSQL")
print("- Traite les données avec Python")
print("- Stocke les résultats dans Redis")
print("- Expose une API simple")

# Créer un docker-compose simple pour l'exercice
simple_compose = '''
version: '3.8'

services:
  app:
    build: .
    ports:
      - "8000:8000"
    depends_on:
      - db
    environment:
      - DATABASE_URL=postgresql://user:pass@db:5432/mydb
    volumes:
      - ./data:/app/data

  db:
    image: postgres:15-alpine
    environment:
      POSTGRES_DB: mydb
      POSTGRES_USER: user
      POSTGRES_PASSWORD: pass
    volumes:
      - postgres_data:/var/lib/postgresql/data

volumes:
  postgres_data:
'''

with open('docker-compose.simple.yml', 'w') as f:
    f.write(simple_compose)

print("\n✅ docker-compose.simple.yml créé")
print("\nCommandes pour l'exercice :")
print("docker-compose -f docker-compose.simple.yml up -d")
print("docker-compose -f docker-compose.simple.yml logs -f")
print("docker-compose -f docker-compose.simple.yml down")

## **Résumé et bonnes pratiques**

### **Points clés à retenir**

In [ ]:
# Résumé des bonnes pratiques Docker
print("📋 Bonnes pratiques Docker pour Data Engineering")
print("\n🔧 Construction d'images :")
print("• Utiliser des images de base officielles et légères (python:3.11-slim)")
print("• Copier les dépendances avant le code pour optimiser le cache")
print("• Utiliser un utilisateur non-root pour la sécurité")
print("• Nettoyer les caches et fichiers temporaires")

print("\n🏗️ Structure des projets :")
print("• Séparer les environnements (dev, test, prod)")
print("• Utiliser des builds multi-stage pour la production")
print("• Documenter les Dockerfiles avec des commentaires")
print("• Versionner les images avec des tags explicites")

print("\n🔄 Développement :")
print("• Monter les volumes pour la persistance des données")
print("• Utiliser Docker Compose pour les environnements complexes")
print("• Implémenter des health checks")
print("• Configurer les logs pour le monitoring")

print("\n⚡ Performance :")
print("• Minimiser le nombre de couches")
print("• Utiliser .dockerignore pour exclure les fichiers inutiles")
print("• Optimiser l'ordre des instructions")
print("• Utiliser des images multi-architecture si nécessaire")

print("\n🎯 Prochaines étapes :")
print("• Intégrer Docker dans votre CI/CD")
print("• Explorer Kubernetes pour l'orchestration")
print("• Apprendre Docker Swarm pour le clustering")
print("• Implémenter la surveillance et le monitoring")

## **Ressources supplémentaires**

### **Documentation et outils**

- **Documentation officielle Docker** : https://docs.docker.com/
- **Docker Hub** : https://hub.docker.com/
- **Docker Compose** : https://docs.docker.com/compose/
- **Bonnes pratiques** : https://docs.docker.com/develop/dev-best-practices/

### **Outils recommandés**

- **Portainer** : Interface graphique pour Docker
- **Docker Desktop** : Environnement de développement
- **Hadolint** : Linter pour Dockerfiles
- **Dive** : Analyser les couches d'images Docker

### **Prochains modules**

- **Jour 6** : Orchestration avec Kubernetes
- **Jour 7** : CI/CD avec Docker
- **Jour 8** : Monitoring et observabilité
- **Jour 9** : Sécurité des containers
- **Jour 10** : Projet final intégré